In [134]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [135]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Instructions
rubric={points}

You will earn points for following these instructions and successfully submitting your work on Gradescope.  

### Group wotk instructions

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2.
  
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


### General submission instructions

- Please **read carefully
[Use of Generative AI policy](https://ubc-cs.github.io/cpsc330-2025W1/syllabus.html#use-of-generative-ai-in-the-course)** before starting the homework assignment. 
- **Run all cells before submitting:** Go to `Kernel -> Restart Kernel and Clear All Outputs`, then select `Run -> Run All Cells`. This ensures your notebook runs cleanly from start to finish without errors.
  
- **Submit your files on Gradescope.**  
   - Upload only your `.ipynb` file **with outputs displayed** and any required output files.
     
   - Do **not** submit other files from your repository.  
   - If you need help, see the [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/).  
- **Check that outputs render properly.**  
   - Make sure all plots and outputs appear in your submission.
     
   - If your `.ipynb` file is too large and doesn't render on Gradescope, also upload a PDF or HTML version so the TAs can view your work.  
- **Keep execution order clean.**  
   - Execution numbers must start at "1" and increase in order.
     
   - Notebooks without visible outputs may not be graded.  
   - Out-of-order or missing execution numbers may result in mark deductions.  
- **Follow course submission guidelines:** Review the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html) for detailed guidance on completing and submitting assignments. 
   
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [136]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [137]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [138]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [139]:
words = ["fast", "easy", "thin", "smart"]

In [140]:
glove_wiki_vectors.most_similar(words[0])

[('slow', 0.795973002910614),
 ('faster', 0.7511823177337646),
 ('pace', 0.7462931871414185),
 ('speed', 0.7133391499519348),
 ('quick', 0.7107294797897339),
 ('easy', 0.6889647245407104),
 ('better', 0.6753882169723511),
 ('slower', 0.673578143119812),
 ('way', 0.6688982248306274),
 ('moving', 0.666520357131958)]

In [141]:
glove_wiki_vectors.most_similar(words[1])

[('easier', 0.8098767995834351),
 ('quick', 0.7886725664138794),
 ('enough', 0.7778581976890564),
 ('better', 0.7729684710502625),
 ('find', 0.7698782682418823),
 ('hard', 0.7667677998542786),
 ('simple', 0.7649385929107666),
 ('difficult', 0.7646346092224121),
 ('good', 0.7633860111236572),
 ('make', 0.7632847428321838)]

In [142]:
glove_wiki_vectors.most_similar(words[2])

[('thick', 0.735665500164032),
 ('flat', 0.6920433044433594),
 ('dark', 0.6790968775749207),
 ('soft', 0.6638060212135315),
 ('thicker', 0.6550114750862122),
 ('thinner', 0.630414605140686),
 ('solid', 0.6274169683456421),
 ('narrow', 0.6247299909591675),
 ('strips', 0.6195363402366638),
 ('lean', 0.6186766028404236)]

In [143]:
glove_wiki_vectors.most_similar(words[3])

[('intelligent', 0.7552732229232788),
 ('sophisticated', 0.6902961134910583),
 ('clever', 0.668724536895752),
 ('pretty', 0.6567649841308594),
 ('kind', 0.6472189426422119),
 ('really', 0.6460521817207336),
 ('incredibly', 0.6443275213241577),
 ('savvy', 0.6280485987663269),
 ('better', 0.6278750896453857),
 ('good', 0.6260865926742554)]

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [144]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [145]:
similarity_results = []
for word1, word2 in word_pairs:

    score = glove_wiki_vectors.similarity(word1, word2)
    similarity_results.append({"word1": word1, "word2": word2, "similarity": score})
    
pd.DataFrame(similarity_results)

,word1,word2,similarity
0,coast,shore,0.700027
1,clothes,closet,0.546276
2,old,new,0.643249
3,smart,intelligent,0.755273
4,dog,cat,0.879808
5,tree,lawyer,0.076719


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [146]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [147]:
test = []

for word in test_words:
 
    is_present = word in glove_wiki_vectors
    
    test.append({
        'Word': word,
        'In Vocabulary': is_present
    })

pd.DataFrame(test)

,Word,In Vocabulary
0,covididiot,False
1,fomo,False
2,frenemies,True
3,anthropause,False
4,photobomb,False
5,selfie,False
6,pxg,False
7,pacg,False
8,cct,True
9,escc,True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [148]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [149]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [150]:
glove_wiki_vectors.similarity("aboriginal", "success")

np.float32(0.14283238)

In [151]:
glove_wiki_vectors.similarity("white", "success")

np.float32(0.351824)

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [152]:
glove_wiki_vectors.similarity("white", "crime")

np.float32(0.38129756)

In [153]:
glove_wiki_vectors.similarity("black", "crime")

np.float32(0.43576178)

In [154]:
glove_wiki_vectors.similarity("man", "strong")

np.float32(0.4733255)

In [155]:
glove_wiki_vectors.similarity("woman", "strong")

np.float32(0.3862984)

In [156]:
analogy("man", "math", "woman")

man : math :: woman : ?


,Analogy word,Score
0,graders,0.648670
1,kindergarten,0.630775
2,literacy,0.627685
3,classroom,0.620775
4,exams,0.619321
5,grades,0.607686
6,mathematics,0.604782
7,maths,0.597781
8,exam,0.590946
9,curriculum,0.586630


In [157]:
analogy("rich", "educated", "poor")

rich : educated :: poor : ?


,Analogy word,Score
0,pupils,0.557099
1,elderly,0.554297
2,uneducated,0.551720
3,schooling,0.544815
4,students,0.540730
5,illiterate,0.537801
6,unemployed,0.534269
7,teachers,0.525769
8,graduates,0.522819
9,indifferent,0.521687


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

1. There are biases like black links more to black than to white, strong links moe to man than to woman, man links more to math and woman links more to kindergarden and literacy, rich people are educated and poor people are uneducated. 
2. If police surveillance system use biased embeddings that link terms like "black" or "uneducated" or "poor" to crime, it will cause unfiarness and scrutiny toward specific populations as well as increases police-civilian conflict. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [158]:
from sklearn.datasets import fetch_20newsgroups

In [159]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [160]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [161]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [162]:
custom_stop_set = {
    'tell', 'idea', 'think', 'know', 'say', 'look', 
    'try', 'want', 'come', 'find', 'make', 'use', 
}


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [163]:
def preprocess_spacy(text):
    doc = nlp(text.lower()) 
    tokens = []
    
    for token in doc:
        if token.like_url:
            processed_token = "url"
        elif token.like_email:
            processed_token = "email"
        elif token.is_digit:
            processed_token = "num"
        else:
            processed_token = token.lemma_

        if processed_token in custom_stop_set:
            continue
        if token.is_stop or token.is_punct or len(processed_token) <= 2:
            continue
        
        is_generic = processed_token in ["url", "email", "num"]

        if is_generic or token.pos_ in ['NOUN', 'ADJ', 'VERB']:
            tokens.append(processed_token)
            
    return " ".join(tokens)


In [164]:
df['text_pp'] = df['text'].apply(preprocess_spacy) 


In [165]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support have newsgroup aspect g...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late spokesman journalist close air space flig...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [166]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [167]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 4
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda.fit_transform(X)

In [168]:
lda.perplexity(X)

np.float64(1979.836327107803)

I chose 4 topics because the target groups can be 4 main groups: computer, religion, sports and politics. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 5

_Type your answer here, replacing this text._

In [169]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

In [170]:
import mglearn

mglearn.tools.print_topics(
    topics=range(4),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           people        
game          url           gun           believe       
team          file          people        thing         
year          email         armenian      time          
play          program       turkish       good          
player        window        government    way           
win           image         state         question      
good          available     right         mean          
season        include       kill          point         
time          run           weapon        life          




In [171]:
topic_labels = {
    0: "sports",
    1: "computers",
    2: "politics",
    3: "religion",
}

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [172]:

num_documents = 5
doc_topic = lda.transform(X[:num_documents])
t = np.argmax(doc_topic, axis=1)

In [173]:
df_head = df.head(num_documents).copy()

df_head['topic_index'] = t
df_head['topic_label'] = df_head['topic_index'].map(topic_labels)
df_head.head()



,text,target,target_name,text_pp,topic_index,topic_label
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,read num u.s.c num wonder help u.s.c num num p...,2,politics
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm bit hard num p...,1,computers
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support have newsgroup aspect g...,3,religion
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...,1,computers
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late spokesman journalist close air space flig...,2,politics


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. It is a supervised machine learning method. It creates a profile for each item (e.g., a movie's genre, director, or actors) and a profile each user based on what they have liked or rated previously. It then finds the similarity between the item's features and predict if the user like the item or not. Unlike collaborative filtering, it does not rely on data from other users; it only looks at the specific user's history and the item's features.

2. Filter Bubbles: Recommender systems are designed to show users what they are likely to agree with or enjoy. This isolates users in a "bubble" of information that only aligns with their existing beliefs, preventing them from exploring something new. 

Bias: If the training data contains biases (e.g., gender or racial bias in hiring data), the recommender system can learn the biases. For example, an algorithm might show high-paying job ads to men, which will be unfair. 

3.Transfer learning is that  a model is first trained on a massive amount of data for a general task (pre-training) and then fine-tuned for a specific, smaller task. Pre-training learns the basic struct/grammar of language context by reading millions of documents (e.g., Wikipedia). Fine-tuning is theprocess for training a model for a specific problem, such as sentiment analysis or medical text classification. 

<!-- END QUESTION -->

<br><br><br><br>

Before submitting your assignment, please make sure you have followed all the instructions in the Submission Instructions section at the top. 

Here is a quick checklist before submitting: 

- [ ] Restart kernel, clear outputs, and run all cells from top to bottom.  
- [ ] `.ipynb` file runs without errors and contains all outputs.  
- [ ] Only `.ipynb` and required output files are uploaded (no extra files).  
- [ ] Execution numbers start at **1** and are in order.  
- [ ] If `.ipynb` is too large and doesn't render on Gradescope, also upload a PDF/HTML version.  
- [ ] Reviewed the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2025W1/docs/homework_instructions.html).  

![](img/eva-well-done.png)